In [ ]:
from bs4 import BeautifulSoup  # Import BeautifulSoup library for web scrapping.
import requests # Import request library use for HTTp request.

import pandas as pd # Import Pandas library for dataframe with read and write CSV.
import schedule # Import schedule library to schedule.

import time # Import time library to schedule.
import datetime  # Import datetime library.
from pathlib import Path # Import path library.

import numpy as np

In [ ]:
# This function will fetch the data from the given url and give it to BeautifulSoup.
def connection(URL):
    r = requests.get(URL) # It will sent the request to the server of given url.
    soup = BeautifulSoup(r.content, 'html5lib') # BeautifulSoup will parse the requests object to HTML5.
    return soup # Returning soup.

In [ ]:
# This function will filter required data from BeautifulSoup and put it in the list.
def get_data(soup):

    # This is the main class where all the properties are.
    table = soup.find('table', attrs = {'class':'browse-table'}) # This is main table tag

    price_final = [] # Defining empty list.
    date_final = [] # Defining empty list.
    address_final = [] # Defining empty list.

    for row in table.findAll('tr'): # For loop will iterate in each properties.
        price = None # Empty variable.
        tag = None # Empty variable.
        tag = row.select('td.browse-cell-date div[title="Last sale price"]') # Get price and store into tag variable.
        if tag:
            price = tag[0].text # Get price from the tag list.
            price = price.replace(' ', '').replace('£', '').replace(',', '').replace('\n', '') # Replace unnecessary space, sign and comma
        price_final.append(price) # Appending the price in the list.

        date = None # Empty variable.
        tag = None # Empty variable.
        tag = row.select('td.browse-cell-date div[title="Last sale date"]') # Get date and store into tag variable.
        if tag:
            date = tag[0].text # Get date from the tag list.
            date = date.replace(',', '').replace('\n', '') # Replace unnecessary space, comma and newline.
            date = datetime.datetime.strptime(date, '%b %Y') # Convert date string into date object.
            date = date.strftime("01-%m-%Y") # Fetch appropriate format of date within the date object.
        date_final.append(date) # Appending the date in the list.

        address = None # Empty variable.
        tag = None # Empty variable.
        tag = row.select('td.browse-cell-address div[class="sold-prices-results-address"]') # Get address and store into tag variable.
        if tag:
            address = tag[0].text # Get address from the tag list.
            address = address.replace('\n', ',') # Replace newline.
            address = " ".join(address.split()) # Replace unnecessary space.
            address = address.replace(', ', ',') # Replace unnecessary space.
            address = address.strip(',') # Remove first and last ,
            address = address.replace(',', ', ') # Give require space.
        address_final.append(address)

    # Create dictionary.
    data_dictionary = {'Address': address_final, 'Sold Price': price_final, 'Sold Date': date_final}

    return data_dictionary # Return data dictionary.

In [ ]:
def create_dataframe(data_dictionary):
    df = pd.DataFrame(data=data_dictionary) # Create DataFrame from dictionary.
    df = df.dropna(how='any',axis=0)  # Remove missing value rows.
    df['Sold Date'] = pd.to_datetime(df['Sold Date']) # String to datetime.
    return df

In [ ]:
# This function is used to call all the above function to scrape the data.
def final_scraper(URL):
    soup = connection(URL) # Call connection function
    data_dictionary = get_data(soup) # Call get_data function
    df = create_dataframe(data_dictionary) # Call create_dataframe function
    return df # Return final DataFrame.

In [ ]:
# This function is used to fetch data from the file
def file_scraper(filename):
    df1 = pd.read_csv(filename) # Load csv file into Dataframe.
    return df1

In [ ]:
# This function is used to merge 2 dataframes
def merge_dataframe(df1, df2):
    df3 = pd.merge(df1, df2, on='Address', how='left') # Merge 2 dataframe(df1, df2) in df3 on address using left join.

    df3['Date Posted'] = pd.to_datetime(df3['Date Posted'])  # String to datetime.
    df3['Sold Date_x'] = pd.to_datetime(df3['Sold Date_x'])  # String to datetime.
    df3['Sold Date_y'] = pd.to_datetime(df3['Sold Date_y'])  # String to datetime.
    #df3.info()  # To Display DataFrame Information.


    df3['Sold Price_y'] = df3['Sold Price_y'].fillna(0).astype(int) # Convert into integer and replace 0 to null.

    df3['Sold Price'] = np.max(df3[['Sold Price_x', 'Sold Price_y']], axis=1) # Combine 2 column(x, y) in one(Sold Price).
    df3['Sold Price'] = df3['Sold Price'].fillna(0).astype(int) # Convert into integer and replace 0 to null.


    df3['Sold Date'] = np.max(df3[['Sold Date_x', 'Sold Date_y']], axis=1) # Combine 2 column(x, y) in one(Sold Date).

    # Drop the additional columns.
    df3 = df3.drop(labels=['Sold Price_x', 'Sold Price_y', 'Sold Date_x', 'Sold Date_y'], axis=1)

    df3['Date Posted'] = pd.to_datetime(df3['Date Posted']) # String to datetime.
    df3['Sold Date'] = pd.to_datetime(df3['Sold Date']) # String to datetime.

    #df3.info() # To Display DataFrame Information.


    df3['Date Posted'] = pd.to_datetime(df3['Date Posted'], format='%d-%m-%Y') # String to datetime with appropriate format.
    df3['Sold Date'] = pd.to_datetime(df3['Sold Date'], format='%d-%m-%Y') # String to datetime with appropriate format.

    df3["Date Posted Year"] = df3["Date Posted"].dt.year # Fetch year from Date Posted.
    df3["Sold Date Year"] = (df3["Sold Date"].dt.year).fillna(0).astype(int)  # Fetch year from Sold Date and fill 0 to null.

    df3["-50%"] = df3["Asking Price"] - ((df3["Asking Price"]*50)/100) # Calculate 50% less
    df3["-50%"] = df3["-50%"].astype(int) # Convert it into integer.

    df3["+50%"] = df3["Asking Price"] + ((df3["Asking Price"]*50)/100) # Calculate 50% more
    df3["+50%"] = df3["+50%"].astype(int) # Convert it into integer.

    # Numpy Array is used for following filter
    # The first one is that the 'Asking Price' date and the 'Sold Price' date are both within the same 12 months.
    # The second filter that the 'Sold Price' needs to be no more or less than +-50%.
    # If these two filters are correct, then the 'Sold Price' is appended to the .csv file, if not, we leave it blank.
    df3['new1'] = np.where(
        ((df3['Date Posted Year'] == df3['Sold Date Year'])&
        ((df3['Sold Price'] >= df3['-50%']) | (df3['Sold Price'] <= df3['+50%']))),
        df3['Sold Price'],
        np.nan
    )

    df3['new1'] = df3['new1'].fillna(0).astype(int) # Convert column new 1 to integer and replace 0 to null

    # drop the additional columns.
    df3 = df3.drop(labels=['Date Posted Year', 'Sold Date Year', '-50%', '+50%', 'Sold Price'], axis=1)

    df3=df3.rename(columns = {'new1':'Sold Price'}) # Rename column new1 to Sold Price

    return df3

In [ ]:
# This function adds Sold Prices to the .csv file
def ChangeExistingFiles():

    filename='Edinburgh_Final.csv'
    URL = ('https://www.zoopla.co.uk/house-prices/edinburgh/hartington-place/?q=5%2F3%20hartington%20place%20Edinburgh&yr=8090623')

    df2 = final_scraper(URL)
    df1 = pd.read_csv(filename) # Load csv file into Dataframe.
    df3 = merge_dataframe(df1, df2)

    # Create new file if file is not exist or if file is exist then it will completely replace old one.
    with open(filename, 'w+', newline='') as f:
        df3.to_csv(f, index=False) # Dateframe to CSV.

    print('Complete')